In [94]:
import pandas as pd

df = pd.read_excel('Time Series.xlsx')

print(df.shape)
df.head(2)

(10885, 4)


,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection


In [95]:
df.Name.unique()

array(['Priyanka', 'Jyoti', 'Deepti', 'Sharan', 'Ravi'], dtype=object)

In [96]:
# df['Start'] = pd.to_datetime(df['Start'])
# df['End'] = pd.to_datetime(df['End'])

In [97]:
df.dtypes

Name                object
Start       datetime64[ns]
End         datetime64[ns]
Activity            object
dtype: object

- In above, columns start and End are having date-time objects with nano-second resolution

In [98]:
df.head()

,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection
2,Jyoti,2023-06-08 08:19:52.620,2023-04-08 05:55:52.620,Updates
3,Priyanka,2023-09-21 15:31:52.620,2023-05-27 10:43:52.620,Reporting
4,Priyanka,2023-10-07 03:31:52.620,2023-04-30 13:07:52.620,Reply to Customers


- **`groupby`**: A function that groups consecutive elements of an iterable based on a key function, returning key-value pairs where each key is associated with its corresponding group.

- below, I'm retrieving start and end times from 1st row of data-frame, printing one of them, and converting to string

In [99]:
df.iloc[0][['Start', 'End']].values

array([Timestamp('2023-10-10 22:43:52.620000'),
       Timestamp('2023-07-20 03:31:52.620000')], dtype=object)

In [100]:
df.iloc[0][['Start', 'End']].values[0]

Timestamp('2023-10-10 22:43:52.620000')

In [101]:
str(df.iloc[0][['Start', 'End']].values[0])

'2023-10-10 22:43:52.620000'

#### Merge Overlapping or Continuous Periods:

- Below code retrieves single group from df.groupby('Name')

In [102]:
i=1
temp_df=0

for group in df.groupby('Name'):
    temp_df = group
    i+=1
    if i==1:
        break

In [103]:
type(temp_df)

tuple

- Below code shows that the temp df just retrieved in above is a tuple with size 2, and its 1st element is string, 2nd element is a data-frame 

In [104]:
print(temp_df[0])
print(type(temp_df[1]))
temp_df[1].head(3)

Sharan
<class 'pandas.core.frame.DataFrame'>


,Name,Start,End,Activity
9,Sharan,2023-06-08 13:07:52.620,2023-09-24 01:07:52.620,Reply to Customers
14,Sharan,2023-09-18 10:43:52.620,2023-07-31 03:31:52.620,Business Development
17,Sharan,2023-09-06 08:19:52.620,2023-05-11 15:31:52.620,Inspection


In [105]:
from itertools import groupby


def merge_periods(group):
    merged_periods = {}
    current_period = group.iloc[0][['Start', 'End']].values
    current_activities = [group.iloc[0]['Activity']]
    
    for i in range(1, len(group)):
        row = group.iloc[i]
        if row['Start'] <= current_period[1]:  # If overlapping or continuous
            current_period[1] = max(current_period[1], row['End'])  # Extend the end time
            current_activities.append(row['Activity'])  # Aggregate activity
        else:
            merged_periods[(str(current_period[0]), str(current_period[1]))] = current_activities
            current_period = row[['Start', 'End']].values
            current_activities = [row['Activity']]
    
    merged_periods[(str(current_period[0]), str(current_period[1]))] = current_activities
    return merged_periods


In [106]:
bot_activity = {}

for group in df.groupby('Name'):
    bot_activity[group[0]] = merge_periods(group[1])

In [107]:
i=0
for key in bot_activity['Deepti'].keys():
    print(type(key))
    print(key)
    print(len(key))
    print(type(key[0]))
    print(key[0])
    if i==0:
        break
# [str((Timestamp('2023-03-29 15:31:52.620000'), Timestamp('2023-10-15 15:31:52.620000')))]

<class 'tuple'>
('2023-08-02 13:07:52.620000', '2023-04-25 10:43:52.620000')
2
<class 'str'>
2023-08-02 13:07:52.620000


- Hence, I created a dictionary(or JSON file) containing bot name, all of their continuous periods and the activities done by them in those continuous periods

- Now, I put this information back in the dataframe

In [108]:
df.columns

Index(['Name', 'Start', 'End', 'Activity'], dtype='object')

In [109]:
df["activity_duration"] = None
df["Activities_done"] = None

In [110]:
for i in range(df.shape[0]):
    temp_dict = bot_activity[df.iloc[i]['Name']]
    for key, value in temp_dict.items():
        duration_starting = key[0]
        duration_ending = key[1]
        duration_starting_ns = pd.to_datetime(duration_starting, format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
        duration_ending_ns = pd.to_datetime(duration_ending, format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
        if (df.iloc[i]['Start'] >= duration_starting_ns) and (df.iloc[i]['End'] <= duration_ending_ns):
            df.loc[i, "activity_duration"] = duration_starting + "_" + duration_ending
            df.loc[i, "Activities_done"] = ", ".join(value)

In [111]:
df.sort_values(by=["Name", "Start"], inplace=True, ascending=[True, True])
df.head()

,Name,Start,End,Activity,activity_duration,Activities_done
4341,Deepti,2023-03-29 15:31:52.620,2023-04-02 08:19:52.620,Business Development,None,None
10807,Deepti,2023-03-29 15:31:52.620,2023-04-20 15:31:52.620,Reply to Customers,None,None
9894,Deepti,2023-03-29 17:55:52.620,2023-05-13 08:19:52.620,Send Email,None,None
10846,Deepti,2023-03-29 17:55:52.620,2023-03-30 22:43:52.620,Fund raising,None,None
181,Deepti,2023-03-29 20:19:52.620,2023-08-11 10:43:52.620,Send Email,None,None


In [112]:
df.isnull().sum()

Name                    0
Start                   0
End                     0
Activity                0
activity_duration    2405
Activities_done      2405
dtype: int64